In [1]:
import ROOT
import matplotlib.pyplot as plt
import numpy as np

Welcome to JupyROOT 6.08/00


### Read csv file with parameters and put it into an array of dictionary

In [2]:
tree = ROOT.TTree("tree","tree");
tree.ReadFile("f_table_wilkinson74.csv");
Z = np.zeros(1,dtype=int)
A = np.zeros(1,dtype=int)
An = np.zeros(20)
tree.SetBranchAddress("Z",Z);
tree.SetBranchAddress("A",A);
tree.SetBranchAddress("An",An);
ftable = []
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    An_c=[]
    Z_c=Z[0]
    A_c=A[0]
    for j in An:
        An_c.append(j)
    ftable.append({"Z": Z_c,"A": A_c,"An": An_c})
np.save("f_table_wilkinson74.npy",ftable)

### Load .npy file and create a function

In [3]:
ftable  = np.load("f_table_wilkinson74.npy",allow_pickle='TRUE')

In [4]:
def f_func(x,p):
    Z = int(p[0])
    E = p[1]-x[0]
    W = E/0.510998918+1 #unit of mec2
    fZ0 = 1./60.*(2*W**4-9*W**2-8)*np.sqrt(W**2-1) +1/4*W*np.log(W+np.sqrt(W**2-1))
    An = ftable[Z-2]["An"]
    An_i=[]
    if (E>0 and E<0.079):
        An_i.append(An[0])
        An_i.append(An[1])
        An_i.append(An[2])
        An_i.append(An[3])
    elif (E>=0.079 and E<0.501):
        An_i.append(An[4])
        An_i.append(An[5])
        An_i.append(An[6])
        An_i.append(An[7])
    elif (E>=0.501 and E<3.162):
        An_i.append(An[8])
        An_i.append(An[9])
        An_i.append(An[10])
        An_i.append(An[11])
    elif (E>=3.162 and E<12.589):
        An_i.append(An[12])
        An_i.append(An[13])
        An_i.append(An[14])
        An_i.append(An[15])
    elif (E>=12.589 and E<25.044):
        An_i.append(An[16])
        An_i.append(An[17])
        An_i.append(An[18])
        An_i.append(An[19])
    #print(An_i)
    S = np.exp(An_i[0]+An_i[1]*np.log(E)+An_i[2]*(np.log(E)**2)+An_i[3]*(np.log(E)**3))
    return fZ0*S          

In [5]:
def f_func_num(x,p):
    return f_func(x,p)*(x[0]-p[2])

In [27]:
def evalEmean(Qb,Sn,ZZ):  
    f_d = ROOT.TF1("f_func",f_func,0.,Qb,2);
    f_d.SetParameter(0,ZZ)
    f_d.SetParameter(1,Qb)
    f_n = ROOT.TF1("f_func_num",f_func_num,0.,Qb,3);
    f_n.SetParameter(0,ZZ)
    f_n.SetParameter(1,Qb)
    f_n.SetParameter(2,Sn)
    return f_n.Integral(Sn,Qb,0.01)/f_d.Integral(Sn,Qb,0.01)

In [28]:
print(evalEmean(14.464,3.63119,49))

1.66962388386


In [38]:
file1 = open("checklist.txt")
count = 0
while True:
    count+=1
    line = file1.readline()
    if not line:
        break
    line = line.strip().split()
    #print(line,count)
    Qb = float(line[3])
    if (Qb>25):
        Qb=25
    Sn = float(line[2])
    if (count != 7):
        Emean = evalEmean(Qb,Sn,int(line[0])-1)
    ediff = abs(Emean -float(line[4]))/float(line[4])*100
    print(int(line[0]),Sn,Qb,ediff)

(6, 4.94631, 13.4369, 0.2345368349849124)
(6, 8.17643, 20.6438, 1.2336438398173637)
(6, 1.21807, 19.0842, 1.7343728418086573)
(6, 0.733572, 22.6844, 1.4097930894961168)
(6, 0.576829, 25, 9.595874971257677)
(7, 2.48885, 8.01023, 0.3879507508171026)
(7, 5.88515, 13.1618, 23.382097404021625)
(7, 2.82823, 11.8061, 0.10947230093205093)
(7, 5.32823, 16.5575, 0.6299460594146766)
(7, 2.16108, 15.7371, 1.5378415293294276)
(7, 1.53843, 21.8464, 1.516538818084261)
(8, 4.14308, 8.67884, 0.86884489885437)
(8, 8.04537, 13.896, 0.2519495065490945)
(8, 3.95564, 12.5234, 0.11599464780480359)
(8, 7.608, 17.9703, 0.18692805352854952)
(8, 3.80546, 17.1699, 1.532630406673734)
(8, 6.85032, 22.4818, 1.7637589301762433)
(8, 2.73298, 22.0991, 1.57295581848837)
(9, 5.23034, 6.48966, 70.71934537202445)
(9, 7.57943, 11.3361, 1.1963474806697416)
(9, 3.81207, 10.9559, 0.23379597984423095)
(10, 10.3643, 10.8181, 139.36522031840113)
(10, 5.20065, 8.43931, 2.0778290074830243)
(10, 8.86892, 13.4962, 0.5662273855391299)